# Training

In [1]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
from src.data_preparation import *
from src.models import *
from src.train_eval import *

from torch.utils.data import DataLoader

In [3]:
train_dataset, val_dataset, test_dataset, norm_stats = prepare_datasets(
    sequence_size=1000,
    stride=25,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    load_if_exists=True
)

🔄 Preparing dataset: Sequence Size: 1000, Stride: 25, Gap Threshold: 0.05
📂 Dataset already downloaded: data\har70.zip
📂 Dataset already extracted in data\har70plus
✅ Successfully loaded HAR70+ dataset (2259597 timestep samples).
✅ Components (HARDataset object, normalization statistics, and split indices) saved to saved_components
✅ Created train, validation, and test datasets (normalized to train set)


In [4]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle= True)
validation_dataloader = DataLoader(val_dataset, batch_size = 64, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size = 64, shuffle= True)

In [5]:
model = HarGRU()
model.load_state_dict(torch.load("models/HarGRU_2025-03-11_19-11-04/HarGRU_epoch100.pth"))
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

HarGRU model loaded on cuda.


In [6]:
training_loss_history, validation_loss_history, accuracy, f1, precision, recall, conf_matrix = train_HAR70_model(
    model, 
    optimizer, 
    train_dataloader, 
    validation_dataloader, 
    num_epochs = 5
)

Epoch [1/5] | Time: 36.30s
(Training) Loss: 0.0009
(Validation) Loss: 0.0013, Accuracy: 0.8686, F1: 0.8788, Precision: 0.8947, Recall: 0.8686
Epoch [2/5] | Time: 32.09s
(Training) Loss: 0.0009
(Validation) Loss: 0.0028, Accuracy: 0.8227, F1: 0.8148, Precision: 0.8455, Recall: 0.8227
Epoch [3/5] | Time: 32.49s
(Training) Loss: 0.0010
(Validation) Loss: 0.0013, Accuracy: 0.8667, F1: 0.8933, Precision: 0.9284, Recall: 0.8667
Epoch [4/5] | Time: 33.12s
(Training) Loss: 0.0009
(Validation) Loss: 0.0017, Accuracy: 0.8585, F1: 0.8873, Precision: 0.9250, Recall: 0.8585
Epoch [5/5] | Time: 35.73s
(Training) Loss: 0.0008
(Validation) Loss: 0.0013, Accuracy: 0.8593, F1: 0.8833, Precision: 0.9148, Recall: 0.8593
✅ Model saved: models\HarGRU_2025-03-11_21-38-23\HarGRU_epoch5.pth


In [7]:
save_training_plots(training_loss_history, validation_loss_history, accuracy, f1, precision, recall)

In [9]:
# Test metrics
loss, accuracy, f1, precision, recall, conf_matrix = evaluate_HAR70_model(model, test_dataloader)
print(f"(Test) Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

(Test) Loss: 0.0012, Accuracy: 0.8654, F1: 0.8937, Precision: 0.9292, Recall: 0.8654
